In [1]:
import csv
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# IPL Team Logos Mapping
team_logos = {
    "RR": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/RR.png?v=2",
    "PBKS": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/PBKS.png?v=2",
    "CSK": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/CSK.png?v=2",
    "DD": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/DD.png?v=2",
    "MI": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/MI.png?v=2",
    "KKR": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/KKR.png?v=2",
    "RCB": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/RCB.png?v=2",
    "DEC": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/DEC.png?v=2",
    "SRH": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/SRH.png?v=2",
    "LSG": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/LSG.png?v=2",
    "GT": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/GT.png?v=2",
    "DC": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/DC.png?v=2",
    "KTK": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/KTK.png?v=2",
    "PWI": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/PWI.png?v=2",
    "GL": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/GL.png?v=2",
    "RPS": "https://ipl-stats-sports-mechanic.s3.ap-south-1.amazonaws.com/ipl/teamlogos/RPS.png?v=2"
}

# Setup Chrome options
options = Options()
options.add_argument("--start-maximized")

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Output folder
folder_name = "IPL Points Tables"
os.makedirs(folder_name, exist_ok=True)

# Loop for 2008 to 2025
for year in range(2008, 2026):
    try:
        url = f"https://www.iplt20.com/points-table/men/{year}"
        driver.get(url)
        time.sleep(5)

        try:
            # Wait for table to load
            table = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, "ih-pt-tab-bg"))
            )
        except TimeoutException:
            print(f"⚠️ Skipping {year}: Points table not found (timeout).")
            continue

        rows = table.find_elements(By.TAG_NAME, "tr")
        headers = [th.text.strip() for th in rows[0].find_elements(By.TAG_NAME, "th") if th.text.strip()]
        skip_last_col = False
        if headers and headers[-1].strip().upper() == "RECENT FORM":
            headers = headers[:-1]
            skip_last_col = True

        team_index = headers.index("Team") if "Team" in headers else 1
        headers.insert(team_index, "Icon URL")

        data = []

        for row in rows[1:]:
            cols = row.find_elements(By.TAG_NAME, "td")
            if not cols or all(col.text.strip() == "" for col in cols):
                continue

            row_data = [col.text.strip() for col in cols if col.text.strip()]
            if not row_data or len(row_data) < len(headers) - 1:
                continue

            team_name = row_data[team_index].split()[0].upper()
            icon_url = team_logos.get(team_name, "")
            row_data.insert(team_index, icon_url)

            if skip_last_col and len(row_data) > len(headers):
                row_data = row_data[:-1]

            data.append(row_data)

        if data:
            file_path = os.path.join(folder_name, f"IPL_Points_Table_{year}.csv")
            with open(file_path, "w", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow(headers)
                writer.writerows(data)
            print(f"✅ {year} Done: {file_path}")
        else:
            print(f"⚠️ {year} Skipped: No data found.")

    except Exception as e:
        print(f"❌ Error {year}: {e}")

# Quit browser
driver.quit()


✅ 2008 Done: IPL Points Tables\IPL_Points_Table_2008.csv
✅ 2009 Done: IPL Points Tables\IPL_Points_Table_2009.csv
✅ 2010 Done: IPL Points Tables\IPL_Points_Table_2010.csv
✅ 2011 Done: IPL Points Tables\IPL_Points_Table_2011.csv
✅ 2012 Done: IPL Points Tables\IPL_Points_Table_2012.csv
✅ 2013 Done: IPL Points Tables\IPL_Points_Table_2013.csv
✅ 2014 Done: IPL Points Tables\IPL_Points_Table_2014.csv
✅ 2015 Done: IPL Points Tables\IPL_Points_Table_2015.csv
✅ 2016 Done: IPL Points Tables\IPL_Points_Table_2016.csv
✅ 2017 Done: IPL Points Tables\IPL_Points_Table_2017.csv
✅ 2018 Done: IPL Points Tables\IPL_Points_Table_2018.csv
✅ 2019 Done: IPL Points Tables\IPL_Points_Table_2019.csv
✅ 2020 Done: IPL Points Tables\IPL_Points_Table_2020.csv
✅ 2021 Done: IPL Points Tables\IPL_Points_Table_2021.csv
✅ 2022 Done: IPL Points Tables\IPL_Points_Table_2022.csv
✅ 2023 Done: IPL Points Tables\IPL_Points_Table_2023.csv
✅ 2024 Done: IPL Points Tables\IPL_Points_Table_2024.csv
✅ 2025 Done: IPL Points Tables\

In [2]:
import csv
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Mapping team abbreviations to full names
team_name_map = {
    'DD': 'Delhi Daredevils',
    'DC': 'Delhi Capitals',
    'PBKS': 'Punjab Kings',
    'KXIP': 'Punjab Kings',
    'CSK': 'Chennai Super Kings',
    'MI': 'Mumbai Indians',
    'KKR': 'Kolkata Knight Riders',
    'RCB': 'Royal Challengers Bangalore',
    'RR': 'Rajasthan Royals',
    'SRH': 'Sunrisers Hyderabad',
    'DEC': 'Deccan Chargers',
    'GL': 'Gujarat Lions',
    'RPS': 'Rising Pune Supergiant',
    'PWI': 'Pune Warriors India',
    'KTK': 'Kochi Tuskers Kerala',
    'GT': 'Gujarat Titans',
    'LSG': 'Lucknow Super Giants'
}

# Setup Chrome options for headless mode
options = Options()
options.add_argument("--headless")  # Classic headless mode
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
)

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Output folder
folder_name = "IPL Points Tables"
os.makedirs(folder_name, exist_ok=True)

# Loop for 2025
for year in range(2008, 2026):
    try:
        url = f"https://www.iplt20.com/points-table/men/{year}"
        driver.get(url)

        # Wait for any table to appear first (ensures JS loaded)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )

        # Scroll to bottom to help JS render content
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        try:
            table = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "ih-pt-tab-bg"))
            )
        except TimeoutException:
            print(f"⚠️ Skipping {year}: Points table not found (timeout after scroll).")
            continue

        rows = table.find_elements(By.TAG_NAME, "tr")
        if not rows:
            print(f"⚠️ {year} Skipped: No rows found.")
            continue

        original_headers = [th.text.strip().upper() for th in rows[0].find_elements(By.TAG_NAME, "th")]
        if not original_headers:
            print(f"⚠️ {year} Skipped: No headers found.")
            continue

        if original_headers[-1].strip().upper() == "RECENT FORM":
            original_headers = original_headers[:-1]

        header_map = {
            "TEAM": "Team",
            "P": "Played",
            "W": "Won",
            "L": "Lost",
            "NRR": "NRR",
            "PTS": "Points"
        }

        headers = [header_map[h] for h in original_headers if h in header_map]
        data = []

        for row in rows[1:]:
            cols = row.find_elements(By.TAG_NAME, "td")
            if not cols:
                continue

            raw_row = [col.text.strip() for col in cols]
            if len(raw_row) > len(original_headers):
                raw_row = raw_row[:-1]

            row_dict = dict(zip(original_headers, raw_row))

            filtered_row = []
            for key in header_map.keys():
                value = row_dict.get(key, "")
                if key == "TEAM":
                    value = team_name_map.get(value.upper(), value)
                filtered_row.append(value)

            if any(filtered_row):
                data.append(filtered_row)

        if data:
            file_path = os.path.join(folder_name, f"IPL_Points_Table_{year}.csv")
            with open(file_path, "w", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow(headers)
                writer.writerows(data)
            print(f"✅ {year} Done: {file_path}")
        else:
            print(f"⚠️ {year} Skipped: No data found.")

    except Exception as e:
        print(f"❌ Error {year}: {e}")

# Quit browser
driver.quit()


✅ 2008 Done: IPL Points Tables\IPL_Points_Table_2008.csv
✅ 2009 Done: IPL Points Tables\IPL_Points_Table_2009.csv
✅ 2010 Done: IPL Points Tables\IPL_Points_Table_2010.csv
✅ 2011 Done: IPL Points Tables\IPL_Points_Table_2011.csv
✅ 2012 Done: IPL Points Tables\IPL_Points_Table_2012.csv
✅ 2013 Done: IPL Points Tables\IPL_Points_Table_2013.csv
✅ 2014 Done: IPL Points Tables\IPL_Points_Table_2014.csv
✅ 2015 Done: IPL Points Tables\IPL_Points_Table_2015.csv
✅ 2016 Done: IPL Points Tables\IPL_Points_Table_2016.csv
✅ 2017 Done: IPL Points Tables\IPL_Points_Table_2017.csv
✅ 2018 Done: IPL Points Tables\IPL_Points_Table_2018.csv
✅ 2019 Done: IPL Points Tables\IPL_Points_Table_2019.csv
✅ 2020 Done: IPL Points Tables\IPL_Points_Table_2020.csv
✅ 2021 Done: IPL Points Tables\IPL_Points_Table_2021.csv
✅ 2022 Done: IPL Points Tables\IPL_Points_Table_2022.csv
✅ 2023 Done: IPL Points Tables\IPL_Points_Table_2023.csv
✅ 2024 Done: IPL Points Tables\IPL_Points_Table_2024.csv
✅ 2025 Done: IPL Points Tables\